In [0]:
import os
!rm -rf *
!rm -rf .kaggle/

!mkdir ~/.kaggle
f = open(".kaggle/kaggle.json", "w")
f.write('{"username":"bkkaggle","key":"bf4e62680116284087ace0484990f87a"}')
f.close()

!git clone https://github.com/bkahn-github/data-science-bowl.git
os.chdir('data-science-bowl/pytorchMultiOutputUnet/')
!git checkout multioutput-unet
!pip install -r ../requirements.txt

!kaggle competitions download -c data-science-bowl-2018
!unzip -q ~/.kaggle/competitions/data-science-bowl-2018/stage1_train.zip -d ~/.kaggle/competitions/data-science-bowl-2018/stage1_train
!unzip -q ~/.kaggle/competitions/data-science-bowl-2018/stage1_test.zip -d ~/.kaggle/competitions/data-science-bowl-2018/stage1_test
!unzip -q ~/.kaggle/competitions/data-science-bowl-2018/stage1_train_labels.csv.zip -d ~/.kaggle/competitions/data-science-bowl-2018/
!unzip -q ~/.kaggle/competitions/data-science-bowl-2018/stage1_sample_submission.csv.zip -d ~/.kaggle/competitions/data-science-bowl-2018/

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1MU9EsOl0wSHzjLiolUzuLLMQHiN_YNgu'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model-20.pt')

In [0]:
!python main.py --subset=True preprocess

In [0]:
!python main.py --subset=False train --epochs=20 --patience=5 --kfolds=1 --batchSize=2 --weights='model-20.pt'

In [0]:
from config import config
from main import visualize

config.SUBSET = True
config.AUGMENT = False

visualize('model-20.pt')
# visualize('model-kfold-0-best.pt')

In [0]:
import os
import logging
import numpy as np

import cv2
import torch
import torchvision

from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

from config import config
from loaders import TrainDataset, augmentation
from model import Unet
from utils import get_kfolds

def postprocess(weights):
    model = Unet()

    model.load_state_dict(torch.load(weights, map_location=lambda storage, location: storage))
    model.eval()

    model.cuda()
  
    kfolds = get_kfolds(2)
  
    dataset = TrainDataset(kfolds[0][0], augmentation=augmentation)
    dataLoader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS)

    for data in dataLoader:
        img, target = data['img'], data['mask']

        x = Variable(img).cuda()
        y = Variable(target).cuda()

        outs = model(x)
        break

    x = x.data.cpu().numpy()
    y = y.data.cpu().numpy()
    outs = outs.data.cpu().numpy()

    return x, y, outs
  
x, y, outs = postprocess('model-20.pt')
# x, y, outs = postprocess('model-kfold-0-best.pt')

# from skimage.morphology import label, binary_dilation, erosion, binary_closing
# from skimage.segmentation import random_walker

# # outs[0][1] *= (255.0/outs[0][1].max())

# mask = outs[0,0]
# contour = outs[0,1]
    
# mask = (mask*255).astype(np.uint8)
# contour = (contour*255).astype(np.uint8)

# _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)
# _, contour = cv2.threshold(contour, 0, 255, cv2.THRESH_OTSU)

# sure_foreground = (contour - mask)
# sure_background = erosion(mask)

# mask_plus_contour = cv2.add(mask, contour)
# mask_plus_contour = cv2.cvtColor(mask_plus_contour, cv2.COLOR_GRAY2RGB)

# unknown = cv2.subtract(sure_background, sure_foreground)

# # Marker labelling
# output = cv2.connectedComponentsWithStats(sure_foreground)
# labels = output[1]
# stats = output[2]
# # Add one to all labels so that sure background is not 0, 0 is considered unknown by watershed
# # this way, watershed can distinguish unknown from the background
# labels = labels + 1
# labels[unknown==255] = 0

# labels = cv2.watershed(mask_plus_contour, labels)
# # labels = random_walker(mask_plus_contour, labels, multichannel=True)   

# labels[labels==-1] = 0
# labels[labels==1] = 0
# labels = labels -1
# labels[labels==-1] = 0

# mean = np.mean(stats[1:,cv2.CC_STAT_AREA])

# for i in range(1, labels.max()):
#      if stats[i, cv2.CC_STAT_AREA] > mean*10 or stats[i, cv2.CC_STAT_AREA] < mean/10:
#         labels[labels==i] = 0

        
# def renumber_labels(label_img):
#     """ Re-number nuclei in a labeled image so the nuclei numbers are unique and consecutive.
#     """
#     new_label = 0
#     for old_label in np.unique(label_img):
#         if not old_label == new_label:
#             label_img[label_img == old_label] = new_label
            
#         new_label += 1
  
#     return label_img

# labels = renumber_labels(labels)
img = np.concatenate((outs[0][0:1].reshape(256, 256, 1), outs[0][1:2].reshape(256, 256, 1), (outs[0][2]).reshape(256, 256, 1)), axis=-1).reshape(256, 256, 3)
truth = np.concatenate((y[0][0:1].reshape(256, 256, 1), y[0][1:2].reshape(256, 256, 1), (y[0][2] > 0).astype(np.uint8).reshape(256, 256, 1)), axis=-1).reshape(256, 256, 3)
x = np.concatenate((x[0][0:1].reshape(256, 256, 1), x[0][1:2].reshape(256, 256, 1), (x[0][2]).reshape(256, 256, 1)), axis=-1).reshape(256, 256, 3)

plt.imshow(img)

In [0]:
img[:,:,1] = img[:,:,1]*10

In [0]:
plt.imshow(img[:,:,:])

In [0]:
import os
import logging

import torch
import torchvision

from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu

from config import config
from loaders import TrainDataset, augmentation
from model import Unet
from utils import get_kfolds

config.SUBSET = False

def postprocess(weights):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model = Unet()

    model.load_state_dict(torch.load(weights, map_location=lambda storage, location: storage))
    model.eval()

    model.to(device)
  
    kfolds = get_kfolds(2)
  
    dataset = TrainDataset(kfolds[0][0], augmentation=augmentation)
    dataLoader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=config.SHUFFLE, num_workers=config.NUM_WORKERS)

    with torch.no_grad():
        for data in dataLoader:
            img, target = data['img'], data['target']

            x = Variable(img).to(device)
            y = Variable(target).to(device)

            outs = model(x)
            break

    x = x.detach().cpu().numpy()
    y = y.detach().cpu().numpy()
    outs = outs.detach().cpu().numpy()
    
    return x, y, outs
  
x, y, outs = postprocess('model-20.pt')
# x, y, outs = postprocess('model-kfold-0-best.pt')

import cv2
import numpy as np
from skimage.morphology import label, binary_dilation, erosion, binary_closing
from skimage.segmentation import random_walker


# outs[0][1] *= (255.0/outs[0][1].max())

mask = outs[0,0]
contour = outs[0,1]
    
mask = (mask*255).astype(np.uint8)
contour = (contour*255).astype(np.uint8)

_, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)
_, contour = cv2.threshold(contour, 0, 255, cv2.THRESH_OTSU)

sure_foreground = (contour - mask)
sure_background = erosion(mask)

mask_plus_contour = cv2.add(mask, contour)
mask_plus_contour = cv2.cvtColor(mask_plus_contour, cv2.COLOR_GRAY2RGB)

unknown = cv2.subtract(sure_background, sure_foreground)

# Marker labelling
output = cv2.connectedComponentsWithStats(sure_foreground)
labels = output[1]
stats = output[2]
# Add one to all labels so that sure background is not 0, 0 is considered unknown by watershed
# this way, watershed can distinguish unknown from the background
labels = labels + 1
labels[unknown==255] = 0
plt.imshow(labels)

labels = cv2.watershed(mask_plus_contour, labels)
# labels = random_walker(mask_plus_contour, labels, multichannel=True)   

labels[labels==-1] = 0
labels[labels==1] = 0
labels = labels -1
labels[labels==-1] = 0

mean = np.mean(stats[1:,cv2.CC_STAT_AREA])

for i in range(1, labels.max()):
     if stats[i, cv2.CC_STAT_AREA] > mean*10 or stats[i, cv2.CC_STAT_AREA] < mean/10:
        labels[labels==i] = 0

        
def renumber_labels(label_img):
    """ Re-number nuclei in a labeled image so the nuclei numbers are unique and consecutive.
    """
    new_label = 0
    for old_label in np.unique(label_img):
        if not old_label == new_label:
            label_img[label_img == old_label] = new_label
        new_label += 1
  
    return label_img
        
labels = renumber_labels(labels)

plt.imshow(labels)

In [0]:
img = np.concatenate((outs[0][0:1].reshape(256, 256, 1), outs[0][1:2].reshape(256, 256, 1), (outs[0][2]).reshape(256, 256, 1)), axis=-1).reshape(256, 256, 3)
truth = np.concatenate((y[0][0:1].reshape(256, 256, 1), y[0][1:2].reshape(256, 256, 1), (y[0][2] > 0).astype(np.uint8).reshape(256, 256, 1)), axis=-1).reshape(256, 256, 3)

In [0]:
plt.imshow(img[:,:,:])

In [0]:
plt.imshow(img[:,:,0] - img[:,:,1])

In [0]:
plt.imshow(truth[:,:,:])

In [0]:
img[:,:,:]

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

uploaded = drive.CreateFile({'title': 'pytorch-multioutput-unet-40-epochs-version-23'})
uploaded.SetContentFile('model-kfold-0-best.pt')

uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
import os
import numpy as np

import torch
import torchvision

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, pooling=True):
        super(ConvBlock, self).__init__()
        self.pooling = pooling
      
        self.conv = nn.Sequential(
          nn.Conv2d(in_ch, out_ch, 3, padding=1),
          nn.BatchNorm2d(out_ch),
          nn.ReLU(inplace=True),
          nn.Dropout2d(0.2),
          nn.Conv2d(out_ch, out_ch, 3, padding=1),
          nn.BatchNorm2d(out_ch),
          nn.ReLU(inplace=True),
          nn.Dropout2d(0.2)
        )
        
    def forward(self, x):
        x = self.conv(x)      
  
        if self.pooling == True:
            x = F.max_pool2d(x, 2)
                            
        return x

class Upsample(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Upsample, self).__init__()

        self.upsample = nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2)
        self.conv = ConvBlock(in_ch, out_ch, pooling=False)

    def forward(self, x1, x2):
        upsample = self.upsample(x1)
        
        cat = torch.cat([x2, upsample], dim=1)
        conv = self.conv(cat)

        return conv

class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()

        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        x = F.sigmoid(x)

        return x

class Unet2(nn.Module):
    def __init__(self):
        super(Unet2, self).__init__()

        self.in_conv = ConvBlock(3, 64, pooling=False)
        self.down_1 = ConvBlock(64, 128)
        self.down_2 = ConvBlock(128, 256)
        self.down_3 = ConvBlock(256, 512)
        self.down_4 = ConvBlock(512, 1024)
        self.up_1 = Upsample(1024, 512)
        self.up_2 = Upsample(512, 256)
        self.up_3 = Upsample(256, 128)
        self.up_4 = Upsample(128, 64)
        
        self.out_conv = OutConv(64, 3)

    def forward(self, x):
        x = x / 255
    
        x1 = self.in_conv(x)
        x2 = self.down_1(x1)
        x3 = self.down_2(x2)
        x4 = self.down_3(x3)
        x5 = self.down_4(x4)
        x = self.up_1(x5, x4)
        x = self.up_2(x, x3)
        x = self.up_3(x, x2)
        x = self.up_4(x, x1)
        
        outputs = self.out_conv(x)

        return outputs

In [0]:
import os
import logging
from tqdm import tqdm
from glob import glob
import argparse

import torch
import torchvision

from torch.utils.data import DataLoader

from config import config
from create_masks import create_masks
from loaders import TrainDataset, x_transforms, target_transforms
from model import Unet
from visualize import show_images
# from metrics import loss
from utils import get_kfolds, calculate_losses, calculate_kfolds_losses, save_model, load_model, EarlyStopping

import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

def loss(inputs, targets):  
  epsilon = 1e-5
  inputs = torch.clamp(inputs.cpu(), epsilon, 1. - epsilon)
  weight = 30 * targets[:,0:1].cpu() + 3 * targets[:,1:2].cpu() + 1 * targets[:,2:3].cpu()
  
  loss = - torch.sum(target.cpu() * weight.cpu() * torch.log(inputs.cpu()) + (1 - targets.cpu()) * torch.log(1 - inputs.cpu()), 1)
#   print(loss)
  
  return loss
  
config.SUBSET = True

epochs = 10
weights = ''
kfolds = 1

kfolds = get_kfolds(kfolds)

if weights != '':
    model = Unet()
    model = load_model(model, weights)

total_kfolds_train_loss = 0
total_kfolds_val_loss = 0

for i, kfold in enumerate(kfolds):
    print('\n')
    logging.info('=' * 50)
    logging.info('Kfold # ' + str(i + 1))

    train_ids, val_ids = kfold[0], kfold[1]

    logging.info('Creating Dataset')
    train = TrainDataset(train_ids, x_transform=x_transforms, target_transforms=target_transforms)
    trainDataloader = DataLoader(train, batch_size=config.BATCH_SIZE, shuffle=config.SHUFFLE, num_workers=config.NUM_WORKERS)
    val = TrainDataset(val_ids, x_transform=x_transforms, target_transforms=target_transforms)
    valDataloader = DataLoader(val, batch_size=config.BATCH_SIZE, shuffle=config.SHUFFLE, num_workers=config.NUM_WORKERS)

    if weights != '' and i == 0:
        model = model
        weights = ''
    else:
        model = Unet()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=config.LR, momentum=0.9)

    early_stopping = EarlyStopping()

    for epoch in range(epochs):
        epoch += 1
        print('\n')
        logging.info('-' * 50)
        logging.info('Epoch # ' + str(epoch))

        total_train_loss = 0
        for data in tqdm(trainDataloader):
            img, target = data['img'], data['target']

            x = img.requires_grad_().to(device)
            y = target.requires_grad_().to(device)

            optimizer.zero_grad()

            outs = model(x)
            train_loss = loss(outs, y)
            
            train_loss.backward(gradient=train_loss)
            total_train_loss += (torch.sum(train_loss.view(-1)) / len(train_loss.view(-1))).item()

            optimizer.step()

        total_val_loss = 0
        with torch.no_grad():
            for data in tqdm(valDataloader):
                img, target = data['img'], data['target']

                x = img.to(device)
                y = target.to(device)

                optimizer.zero_grad()

                outs = model(x)
                val_loss = loss(outs, y)
                total_val_loss += (torch.sum(val_loss.view(-1)) / len(val_loss.view(-1))).item()

        message, train_loss, val_loss = calculate_losses(total_train_loss, total_val_loss, train_ids, val_ids, epoch)
        print(message)

        total_kfolds_train_loss += train_loss
        total_kfolds_val_loss += val_loss

        action = early_stopping.evaluate(model, val_loss, epoch, config.PATIENCE)

        if action == 'save':
            save_model(model, i)
        elif action == 'stop':
            break
        else:
            continue

message = calculate_kfolds_losses(total_kfolds_train_loss, total_kfolds_val_loss, config.KFOLDS, config.EPOCHS)
print(message)

In [0]:
# !ls
os.chdir('./pytorchMultiOutputUnet/')

In [0]:
# (torch.sum(temp.view(-1)) / len(temp.view(-1))).item()
train_loss

In [0]:
weights = 'model-kfold-0-best.pt'

model = Unet()
model.load_state_dict(torch.load(weights))
model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

kfolds = get_kfolds(2)

dataset = TrainDataset(kfolds[0][0], x_transform=x_transforms, target_transforms=target_transforms)
dataLoader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=config.SHUFFLE, num_workers=config.NUM_WORKERS)

with torch.no_grad():
    for data in dataLoader:
        img, target = data['img'], data['target']

        x = Variable(img).to(device)
        y = Variable(target).to(device)

        outs = model(x)
        break

y = y.detach().cpu().numpy()
outs = outs.detach().cpu().numpy()

In [0]:
plt.imshow(outs[0].reshape(256, 256, 3))

In [0]:
import os
import glob
from tqdm import tqdm
from PIL import Image
from imageio import imwrite
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage
import skimage.morphology
from config import config

def get_edges(img):
    img = skimage.morphology.binary_dilation(img, selem=np.ones((5,5))).astype(np.uint8)
    return img
  
def get_sizes(mask_folder):
    mask = glob.glob(os.path.join(mask_folder, 'masks/*'))[0]
    img = Image.open(mask)
    img = np.asarray(img)

    return img.shape

def create_masks(root_folder, stage_number, stage_section, output_folder, mode, subset=False):
    stage_folder = os.path.join(root_folder, 'stage' + stage_number + '_' + stage_section) 
    os.makedirs(stage_folder + '_' + mode, exist_ok=True)

    if subset:
        masks_folder = glob.glob(os.path.join(stage_folder, '*'))[:20]
    else:
        masks_folder = glob.glob(os.path.join(stage_folder, '*'))        
    
    for mask_folder in tqdm(masks_folder):
        mask_id = mask_folder.split('/')[-1]

        size = get_sizes(mask_folder)
        
        masks = np.zeros(size)
        masks_with_edges = np.zeros(size)

        for mask in glob.glob(os.path.join(mask_folder, 'masks/*')):
            img = Image.open(mask)
            img = np.asarray(img)
            img = img / 255.0

            img_with_edges = get_edges(img)
            
            masks = np.add(masks, img)
            masks_with_edges = np.add(masks_with_edges, img_with_edges)

        target = np.zeros((size[0], size[1], 3))
        
        target[:,:,0] = masks == 1
        target[:,:,1] = masks_with_edges == 2
        target[:,:,2] = masks == 0
        
        target *= 255
        target = target.astype(np.uint8)

#         return target
        output_path = os.path.join(stage_folder + '_' + mode, mask_id + '.png')        
        imwrite(output_path, target)
        
test = create_masks(config.ROOT_FOLDER, config.STAGE, 'train', config.TARGETS_FOLDER, 'masks', config.SUBSET)

In [0]:
plt.imshow(test[:,:,0])

In [0]:
plt.imshow(test[:,:,1])

In [0]:
!kill -9 -1